In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
from collections import Counter
from sklearn import preprocessing
import scipy

In [2]:
ls data

crime_astros_finalA.csv*  crime_rockets_finalA.csv*  crime_texans_finalA.csv*
crime_astros_finalB.csv*  crime_rockets_finalB.csv*  crime_texans_finalB.csv*
crime_astros_final.csv*   crime_rockets_final.csv*   crime_texans_final.csv*
crime_dynamo_finalA.csv*  crime_ru_finalB.csv*       crime_uh_finalA.csv*
crime_dynamo_finalB.csv*  crime_ru_final.csv*        crime_uh_finalB.csv*
crime_dynamo_final.csv*   crime_ru_final.csvA.csv*   crime_uh_final.csv*


In [3]:
path = 'data/crime_uh_finalA.csv'

In [4]:
df = pd.read_csv(path)
#df.drop(['date'],axis=1,inplace=True)
df = df.set_index('date').sort_index(ascending=True)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16826 entries, 2010-01-01 to 2017-12-31
Data columns (total 8 columns):
OffenseType     16826 non-null object
Premise         16826 non-null object
hour            16826 non-null int64
weekday         16826 non-null object
month           16826 non-null object
year            16826 non-null int64
dist_stadium    16826 non-null float64
game            16826 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 1.2+ MB


## 1 mile radius

In [6]:
one_mile =df.dist_stadium <= 1
df = df[one_mile]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6650 entries, 2010-01-01 to 2017-12-30
Data columns (total 8 columns):
OffenseType     6650 non-null object
Premise         6650 non-null object
hour            6650 non-null int64
weekday         6650 non-null object
month           6650 non-null object
year            6650 non-null int64
dist_stadium    6650 non-null float64
game            6650 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 467.6+ KB


# Create features
- holiday
- season


In [8]:
# import pandas as pd
# from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
# dr = pd.date_range(start='2010-01-01', end='2017-12-31')

# cal = calendar()
# holidays = cal.holidays(start=dr.min(), end=dr.max())

# df['holiday'] = df.index.isin(holidays)
# df['holiday'] = df['holiday'].astype(int)  # change to 0, or 1

# df.head()

In [9]:
def day_feature(df):
    mo = [6,7,8,9,10,11]  # morning, sunrise to 11
    af = [12,13,14,15,16]  # afternoon to fiveish
    ev = [17,18,19,20]   # evening to  sunset
    ni = [21,22,23,0,1,2,3,4,5]  # night, sunset to sunrise
    if df.hour in mo:
        val = 'Morning'
    elif df.hour in af:
        val = 'Afternoon'
    elif df.hour in ev:
        val = 'Evening'
    else:
        val = 'Night'
    return val

df['part_day'] = df.apply(day_feature,axis=1)

In [10]:
df.head()

,OffenseType,Premise,hour,weekday,month,year,dist_stadium,game,part_day
date,,,,,,,,,
2010-01-01,Robbery,bank prk,4,Friday,Jan,2010,0.359544,No Game,Night
2010-01-01,Burglary,house,12,Friday,Jan,2010,0.527477,No Game,Afternoon
2010-01-01,Theft,house,12,Friday,Jan,2010,0.527477,No Game,Afternoon
2010-01-01,Aggravated Assault,street,18,Friday,Jan,2010,0.928047,No Game,Evening
2010-01-01,Aggravated Assault,house,21,Friday,Jan,2010,0.527477,No Game,Night


In [11]:
def season_feature(df):
    '''
    spring (March, April, May), 
    summer (June, July, August), 
    autumn (September, October, November)
    winter (December, January, February).
    '''
    sp = ['Mar','Apr','May']   # spring
    su = ['Jun','Jul','Aug']   # summer
    au = ['Sep','Oct','Nov'] # autumn/fall
    wi = ['Dec','Jan','Feb']  # winter
    if df.month in sp:
        val = 'Spring'
    elif df.month in su:
        val = 'Summer'
    elif df.month in au:
        val = 'Autumn'
    else:
        val = 'Winter'
    return val

df['season'] = df.apply(season_feature,axis=1)

In [12]:
df.head()

,OffenseType,Premise,hour,weekday,month,year,dist_stadium,game,part_day,season
date,,,,,,,,,,
2010-01-01,Robbery,bank prk,4,Friday,Jan,2010,0.359544,No Game,Night,Winter
2010-01-01,Burglary,house,12,Friday,Jan,2010,0.527477,No Game,Afternoon,Winter
2010-01-01,Theft,house,12,Friday,Jan,2010,0.527477,No Game,Afternoon,Winter
2010-01-01,Aggravated Assault,street,18,Friday,Jan,2010,0.928047,No Game,Evening,Winter
2010-01-01,Aggravated Assault,house,21,Friday,Jan,2010,0.527477,No Game,Night,Winter


## Groupby date and get values

In [13]:
#df.groupby(df.index)['part_day','hour','Premise','OffenseType'].agg(lambda x: scipy.stats.mode(x, axis=None)[0][0]).head()

df['part_day_mode'] = df.groupby(df.index)['part_day'].agg(lambda x: scipy.stats.mode(x, axis=None)[0][0])
df['hour_mode'] = df.groupby(df.index)['hour'].agg(lambda x: scipy.stats.mode(x, axis=None)[0][0])
df['premise_mode'] = df.groupby(df.index)['Premise'].agg(lambda x: scipy.stats.mode(x, axis=None)[0][0])
df['offenseType_mode'] = df.groupby(df.index)['OffenseType'].agg(lambda x: scipy.stats.mode(x, axis=None)[0][0])


/home/frank/miniconda3/envs/springboard/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [14]:
df.head()

,OffenseType,Premise,hour,weekday,month,year,dist_stadium,game,part_day,season,part_day_mode,hour_mode,premise_mode,offenseType_mode
date,,,,,,,,,,,,,,
2010-01-01,Robbery,bank prk,4,Friday,Jan,2010,0.359544,No Game,Night,Winter,Afternoon,12,house,Aggravated Assault
2010-01-01,Burglary,house,12,Friday,Jan,2010,0.527477,No Game,Afternoon,Winter,Afternoon,12,house,Aggravated Assault
2010-01-01,Theft,house,12,Friday,Jan,2010,0.527477,No Game,Afternoon,Winter,Afternoon,12,house,Aggravated Assault
2010-01-01,Aggravated Assault,street,18,Friday,Jan,2010,0.928047,No Game,Evening,Winter,Afternoon,12,house,Aggravated Assault
2010-01-01,Aggravated Assault,house,21,Friday,Jan,2010,0.527477,No Game,Night,Winter,Afternoon,12,house,Aggravated Assault


In [15]:
df.apply(lambda x:sum(x.isnull()),axis=0)

OffenseType         0
Premise             0
hour                0
weekday             0
month               0
year                0
dist_stadium        0
game                0
part_day            0
season              0
part_day_mode       0
hour_mode           0
premise_mode        0
offenseType_mode    0
dtype: int64

In [16]:
df.apply(lambda x: len(x.unique()))

OffenseType           7
Premise              62
hour                 24
weekday               7
month                12
year                  8
dist_stadium        486
game                  3
part_day              4
season                4
part_day_mode         4
hour_mode            24
premise_mode         45
offenseType_mode      7
dtype: int64

In [17]:
print(list(df.columns))

['OffenseType', 'Premise', 'hour', 'weekday', 'month', 'year', 'dist_stadium', 'game', 'part_day', 'season', 'part_day_mode', 'hour_mode', 'premise_mode', 'offenseType_mode']


In [18]:
df = df[['OffenseType', 'weekday', 'month', 'year', 'dist_stadium', 'game', 'season', 'part_day_mode', 'hour_mode', 'premise_mode', 'offenseType_mode']]
df.head()

,OffenseType,weekday,month,year,dist_stadium,game,season,part_day_mode,hour_mode,premise_mode,offenseType_mode
date,,,,,,,,,,,
2010-01-01,Robbery,Friday,Jan,2010,0.359544,No Game,Winter,Afternoon,12,house,Aggravated Assault
2010-01-01,Burglary,Friday,Jan,2010,0.527477,No Game,Winter,Afternoon,12,house,Aggravated Assault
2010-01-01,Theft,Friday,Jan,2010,0.527477,No Game,Winter,Afternoon,12,house,Aggravated Assault
2010-01-01,Aggravated Assault,Friday,Jan,2010,0.928047,No Game,Winter,Afternoon,12,house,Aggravated Assault
2010-01-01,Aggravated Assault,Friday,Jan,2010,0.527477,No Game,Winter,Afternoon,12,house,Aggravated Assault


In [19]:
cdf = df.groupby(df.index).agg(
    {'OffenseType':'count',
     'weekday':'first',
     'month':'first',
     'year': 'first',
     'dist_stadium':'median',
     'season':'first',
     'part_day_mode':'first',
     'hour_mode':'first',
     'premise_mode':'first',
     'offenseType_mode':'first',
    'game':'first'})

cdf.head()

,OffenseType,weekday,month,year,dist_stadium,season,part_day_mode,hour_mode,premise_mode,offenseType_mode,game
date,,,,,,,,,,,
2010-01-01,5,Friday,Jan,2010,0.527477,Winter,Afternoon,12,house,Aggravated Assault,No Game
2010-01-02,2,Saturday,Jan,2010,0.713069,Winter,Morning,10,apartment,Burglary,No Game
2010-01-03,1,Sunday,Jan,2010,0.648625,Winter,Morning,7,apartment,Aggravated Assault,No Game
2010-01-04,6,Monday,Jan,2010,0.538842,Winter,Evening,17,street,Theft,No Game
2010-01-06,2,Wednesday,Jan,2010,0.594522,Winter,Afternoon,1,rest,Robbery,No Game


In [20]:
df.apply(lambda x: len(x.unique()))

OffenseType           7
weekday               7
month                12
year                  8
dist_stadium        486
game                  3
season                4
part_day_mode         4
hour_mode            24
premise_mode         45
offenseType_mode      7
dtype: int64

In [21]:
# rename column
cdf.rename(columns={'OffenseType': 'crime_total','dist_stadium':'dist_stadium_meadian','offenseType_mode':'offense_mode'}, inplace=True)

In [22]:
cdf.head()

,crime_total,weekday,month,year,dist_stadium_meadian,season,part_day_mode,hour_mode,premise_mode,offense_mode,game
date,,,,,,,,,,,
2010-01-01,5,Friday,Jan,2010,0.527477,Winter,Afternoon,12,house,Aggravated Assault,No Game
2010-01-02,2,Saturday,Jan,2010,0.713069,Winter,Morning,10,apartment,Burglary,No Game
2010-01-03,1,Sunday,Jan,2010,0.648625,Winter,Morning,7,apartment,Aggravated Assault,No Game
2010-01-04,6,Monday,Jan,2010,0.538842,Winter,Evening,17,street,Theft,No Game
2010-01-06,2,Wednesday,Jan,2010,0.594522,Winter,Afternoon,1,rest,Robbery,No Game


In [23]:
path

'data/crime_uh_finalA.csv'

In [24]:
cdf.to_csv('data/crime_uh_finalB.csv')